# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Jun 27 2022  9:29AM  240477        10  0085799670  ISDIN Corporation   
1  Jun 27 2022  9:29AM  240477        10  0085799674  ISDIN Corporation   
2  Jun 27 2022  9:29AM  240477        10  0085799779  ISDIN Corporation   
3  Jun 27 2022  9:29AM  240477        10  0085799775  ISDIN Corporation   
4  Jun 27 2022  9:29AM  240477        10  0085799781  ISDIN Corporation   
5  Jun 27 2022  9:29AM  240477        10  0085799780  ISDIN Corporation   
6  Jun 27 2022  9:29AM  240477        10  0085799786  ISDIN Corporation   
7  Jun 27 2022  9:29AM  240477        10  0085799783  ISDIN Corporation   
8  Jun 27 2022  9:29AM  240477        10  0085799784  ISDIN Corporation   
9  Jun 27 2022  9:29AM  240477        10  0085799787  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
17  240475       Released         18
18  240476       Released          1
19  240477       Released         52
20  240478       Released          3
21  240480       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
240475                NaN      18.0
240476                NaN       1.0
240477                NaN      52.0
240478                NaN       3.0
240480                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
240454               30.0       2.0
240455                0.0       4.0
240459                0.0      48.0
240460               39.0      12.0
240462                0.0      36.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
240454                 30         2
240455                  0         4
240459                  0        48
240460                 39        12
240462                  0        36

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               240454         30         2
1               240455          0         4
2               240459          0        48
3               240460         39        12
4               240462          0        36

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing  Released
0               240454        30         2
1               240455                   4
2               240459                  48
3               240460        39        12
4               240462                  36

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse           Customer
0    Jun 27 2022  9:29AM  240477        10  ISDIN Corporation
52   Jun 27 2022  9:28AM  240480        20     Exact-Rx, Inc.
53   Jun 27 2022  9:26AM  240474        10  ISDIN Corporation
106  Jun 27 2022  9:25AM  240475        20  Alumier Labs Inc.
124  Jun 27 2022  9:24AM  240478        10     Citieffe, Inc.
127  Jun 27 2022  9:23AM  240476        16    SHL Pharma, LLC
128  Jun 27 2022  9:14AM  240471        10  ISDIN Corporation
185  Jun 27 2022  9:11AM  240472        16    SHL Pharma, LLC
186  Jun 27 2022  9:05AM  240470        10  ISDIN Corporation
236  Jun 27 2022  9:01AM  240467        10  ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse           Customer Executing  \
0  Jun 27 2022  9:29AM  240477        10  ISDIN Corporation             
1  Jun 27 2022  9:28AM  240480        20     Exact-Rx, Inc.             
2  Jun 27 2022  9:26AM  240474        10  ISDIN Corporation             
3  Jun 27 2022  9:25AM  240475        20  Alumier Labs Inc.             
4  Jun 27 2022  9:24AM  240478        10     Citieffe, Inc.             
5  Jun 27 2022  9:23AM  240476        16    SHL Pharma, LLC             
6  Jun 27 2022  9:14AM  240471        10  ISDIN Corporation             
7  Jun 27 2022  9:11AM  240472        16    SHL Pharma, LLC             
8  Jun 27 2022  9:05AM  240470        10  ISDIN Corporation             
9  Jun 27 2022  9:01AM  240467        10  ISDIN Corporation             

   Released  
0        52  
1         1  
2        53  
3        18  
4         3  
5         1  
6        57  
7         1  
8        50  
9        49

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse           Customer  Released  \
0  Jun 27 2022  9:29AM  240477        10  ISDIN Corporation        52   
1  Jun 27 2022  9:28AM  240480        20     Exact-Rx, Inc.         1   
2  Jun 27 2022  9:26AM  240474        10  ISDIN Corporation        53   
3  Jun 27 2022  9:25AM  240475        20  Alumier Labs Inc.        18   
4  Jun 27 2022  9:24AM  240478        10     Citieffe, Inc.         3   
5  Jun 27 2022  9:23AM  240476        16    SHL Pharma, LLC         1   
6  Jun 27 2022  9:14AM  240471        10  ISDIN Corporation        57   
7  Jun 27 2022  9:11AM  240472        16    SHL Pharma, LLC         1   
8  Jun 27 2022  9:05AM  240470        10  ISDIN Corporation        50   
9  Jun 27 2022  9:01AM  240467        10  ISDIN Corporation        49   

  Executing  
0            
1            
2            
3            
4            
5            
6            
7            
8            
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jun 27 2022  9:29AM  240477        10  ISDIN Corporation        52   
1  Jun 27 2022  9:28AM  240480        20     Exact-Rx, Inc.         1   
2  Jun 27 2022  9:26AM  240474        10  ISDIN Corporation        53   
3  Jun 27 2022  9:25AM  240475        20  Alumier Labs Inc.        18   
4  Jun 27 2022  9:24AM  240478        10     Citieffe, Inc.         3   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse           Customer  Released  \
0  Jun 27 2022  9:29AM  240477        10  ISDIN Corporation        52   
1  Jun 27 2022  9:28AM  240480        20     Exact-Rx, Inc.         1   
2  Jun 27 2022  9:26AM  240474        10  ISDIN Corporation        53   
3  Jun 27 2022  9:25AM  240475        20  Alumier Labs Inc.        18   
4  Jun 27 2022  9:24AM  240478        10     Citieffe, Inc.         3   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Jun 27 2022  9:29AM  240477        10  ISDIN Corporation        52   
1  Jun 27 2022  9:28AM  240480        20     Exact-Rx, Inc.         1   
2  Jun 27 2022  9:26AM  240474        10  ISDIN Corporation        53   
3  Jun 27 2022  9:25AM  240475        20  Alumier Labs Inc.        18   
4  Jun 27 2022  9:24AM  240478        10     Citieffe, Inc.         3   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         2645153       421       39.0
12          480927        32       16.0
16          480948         2        0.0
19          480909         6       30.0
20          480955        19        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  2645153       421       39.0
1        12   480927        32       16.0
2        16   480948         2        0.0
3        19   480909         6       30.0
4        20   480955        19        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10       421       39.0
1        12        32       16.0
2        16         2        0.0
3        19         6       30.0
4        20        19        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  421.0
1        12  Released   32.0
2        16  Released    2.0
3        19  Released    6.0
4        20  Released   19.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12   16    19    20
Status                                 
Executing   39.0  16.0  0.0  30.0   0.0
Released   421.0  32.0  2.0   6.0  19.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12   16    19    20
0          Executing   39.0  16.0  0.0  30.0   0.0
1           Released  421.0  32.0  2.0   6.0  19.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12   16    19    20
0  Executing   39.0  16.0  0.0  30.0   0.0
1   Released  421.0  32.0  2.0   6.0  19.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()